In [264]:
from sklearn.ensemble import RandomForestRegressor

In [318]:
def read_and_concat(file_list):
    dfs = [pd.read_csv(f) for f in file_list]
    return pd.concat(dfs, ignore_index=True)

In [319]:
import glob
import pandas as pd

file_list_2022 = glob.glob("2022/*.csv")
file_list_2023 = glob.glob("2023/*.csv")
file_list_2024 = glob.glob("2024/*.csv")
file_list_2025 = glob.glob("2025/*.csv")

In [320]:
df_2022 = read_and_concat(file_list_2022)
df_2023 = read_and_concat(file_list_2023)
df_2024 = read_and_concat(file_list_2024)

big_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)
big_df = big_df.drop(["FullName"], axis=1)
big_df = big_df.dropna(subset=["FinishingPos"])
big_df = pd.get_dummies(big_df)

In [321]:
X_train = big_df.drop(["FinishingPos"], axis=1)
y_train = big_df["FinishingPos"]

In [322]:
test_df = read_and_concat(file_list_2025)
test_df = test_df.drop(["FullName"], axis=1)
test_df = test_df.dropna(subset=["FinishingPos"])
test_df = pd.get_dummies(test_df)

In [323]:
X_test = test_df.drop(["FinishingPos"], axis=1)
Y_test = test_df["FinishingPos"]

In [324]:
# After creating one-hot encoded DataFrames:
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

In [367]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    "n_estimators" : [i for i in range(1, 1001, 10)],
    "max_depth" : [i for i in range(1,101)],
    "min_samples_split" : [i for i in range(2,101)],
    "min_samples_leaf" : [i for i in range(1,101)],
    "max_features" : ["sqrt", "log2", None],
    "bootstrap" : [True],
    "max_samples" : [0.5, 0.6, 0.7, 0.8, 0.9],
    "random_state" : [30]
}

rfr = RandomForestRegressor(random_state=30)
rfr = RandomizedSearchCV(
    estimator=rfr,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    verbose=2,
    n_jobs=-1,
    scoring="r2"
)

In [368]:
rfr.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


,estimator,RandomForestR...ndom_state=30)
,param_distributions,"{'bootstrap': [True], 'max_depth': [1, 2, ...], 'max_features': ['sqrt', 'log2', ...], 'max_samples': [0.5, 0.6, ...], ...}"
,n_iter,100
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [369]:
Y_pred = rfr.predict(X_test)

In [370]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(mean_absolute_error(Y_pred, Y_test))
print(mean_squared_error(Y_pred, Y_test))
print(r2_score(Y_pred, Y_test))

2.2961373830102727
9.803257936593248
0.6284391055419876


In [371]:
for file in file_list_2025:
    spa = pd.read_csv(file)
    spa = spa.drop(["FullName"], axis=1)
    spa = spa.dropna(subset=["FinishingPos"])
    spa = pd.get_dummies(spa)
    spa_X = spa.drop(["FinishingPos"], axis=1)
    spa_Y = spa["FinishingPos"]

    X_train, spa_X = X_train.align(spa_X, join='left', axis=1, fill_value=0)
    spa_Y_pred = hgbr.predict(spa_X)

    print(r2_score(spa_Y, spa_Y_pred), file)

0.6797840726681988 2025\Barcelona_race.csv
0.7753610343519604 2025\Imola_race.csv
0.8588368619602593 2025\Jeddah_race.csv
0.6801956665475755 2025\Melbourne_race.csv
0.8640840246484085 2025\Miami_race.csv
0.6608937952402469 2025\Monaco_race.csv
0.7844245095749751 2025\Montreal_race.csv
0.6803525459683339 2025\Sakhir_race.csv
0.7101355925853186 2025\Shanghai_race.csv
0.31396475545595326 2025\Silverstone_race.csv
0.5379711402443756 2025\Spa-Francorchamps_race.csv
0.694013558484269 2025\Spielberg_race.csv
0.8731883071942839 2025\Suzuka_race.csv


In [293]:
spa = pd.read_csv("2025/Silverstone_race.csv")
spa = spa.drop(["FullName"], axis=1)
spa = spa.dropna(subset=["FinishingPos"])
spa = pd.get_dummies(spa)
spa_X = spa.drop(["FinishingPos", "GapToPole"], axis=1)
spa_Y = spa["FinishingPos"]

X_train, spa_X = X_train.align(spa_X, join='left', axis=1, fill_value=0)
spa_Y_pred = rfr.predict(spa_X)

print(r2_score(spa_Y, spa_Y_pred), file)

0.33260812030075193 2025\Suzuka_race.csv


In [294]:
driver_preds = {}
for i, driver in enumerate(spa_X.iterrows()):
    driver_preds[driver[1]["DriverNumber"]] = [spa_Y_pred[i], spa_Y[i]]

In [295]:
sorted_items = sorted(driver_preds.items(), key=lambda item: item[1])

In [297]:
for i, driver in enumerate(sorted_items):
    print(f"{driver_names[driver[0]]} - {i + 1} - {driver[1][0]} - {driver[1][1]}")

Max Verstappen - 1 - 2.175 - 5.0
Lando Norris - 2 - 2.595 - 1.0
Oscar Piastri - 3 - 2.775 - 2.0
George Russell - 4 - 3.32 - 10.0
Lewis Hamilton - 5 - 3.345 - 4.0
Charles Leclerc - 6 - 3.575 - 14.0
Yuki Tsunoda - 7 - 7.84 - 15.0
Fernando Alonso - 8 - 8.27 - 9.0
Oliver Bearman - 9 - 9.115 - 11.0
Pierre Gasly - 10 - 10.26 - 6.0
Carlos Sainz - 11 - 11.265 - 12.0
Estaban Ocon - 12 - 12.695 - 13.0
Alex Albon - 13 - 12.77 - 8.0
Lance Stroll - 14 - 13.275 - 7.0
Nico Hulkenberg - 15 - 14.355 - 3.0
Kimi Antonelli - 16 - 17.82 - 16.0
Franco Colapinto - 17 - 17.915 - 20.0
Liam Lawson - 18 - 18.1 - 19.0
Gabriel Bortoleto - 19 - 18.95 - 18.0
Isack Hadjar - 20 - 19.095 - 17.0


In [221]:
driver_names = {
    4 : "Lando Norris",
    1 : "Max Verstappen",
    81 : "Oscar Piastri",
    16 : "Charles Leclerc",
    22 : "Yuki Tsunoda",
    23 : "Alex Albon",
    63 : "George Russell",
    6 : "Isack Hadjar",
    5 : "Gabriel Bortoleto",
    10 : "Pierre Gasly",
    30 : "Liam Lawson",
    31 : "Estaban Ocon",
    87 : "Oliver Bearman",
    12 : "Kimi Antonelli",
    44 : "Lewis Hamilton",
    14 : "Fernando Alonso",
    18 : "Lance Stroll",
    43 : "Franco Colapinto",
    55 : "Carlos Sainz",
    27 : "Nico Hulkenberg",
    7 : "Jack Doohan"
}